**genetic algorithm applied to neural network weight optimization for a specific machine learning task**

In [1]:
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

from keras.callbacks import EarlyStopping, TensorBoard
from keras.layers import Dense
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler

from tqdm import tqdm
import numpy as np
import random
import pickle as pkl

import matplotlib.pyplot as plt

<h1 id="dataset" style="color:#024434; background:#f4684b; border:0.5px dotted;">
    <center>Dataset
        <a class="anchor-link" href="#dataset" target="_self">¶</a>
    </center>
</h1>

**The following code is focused on data preprocessing, particularly for preparing a dataset for a machine learning or deep learning task. **




In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


This code appears to be related to preprocessing a dataset for a machine learning task. It reads a dataset from a CSV file, encodes categorical features using one-hot encoding and label encoding, scales the data with standardization, and then splits the dataset into training, validation, and testing sets.

In [3]:
dataset = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Phd/Genetic Approach/Tonsil_updatedata.csv')
y_train = dataset['Vital status recode']

x_train = dataset.drop(labels =['Year of diagnosis','Sex','Grade','SEER cause-specific death classification','RX Summ--Surg Prim Site','Patient ID','Combined Summary Stage','Diagnostic Confirmation','Behavior code ICD-O-2','Histology recode  broad groupings'],axis=1)

categorical_features = dataset.select_dtypes(include=['object', 'int']).columns.tolist()
for feature in categorical_features:
    unique_values = dataset[feature].nunique()

    print(f"Feature '{feature}' has {unique_values} unique values.")



Feature 'Age' has 14 unique values.
Feature 'Race recode' has 3 unique values.
Feature 'Year of diagnosis' has 12 unique values.
Feature 'Sex' has 2 unique values.
Feature 'Primary Site  labeled' has 4 unique values.
Feature 'ICD-O-3 Hist behav' has 27 unique values.
Feature 'Laterality' has 5 unique values.
Feature 'Site recode - rare tumors' has 5 unique values.
Feature 'AYA site recode ' has 2 unique values.
Feature 'Derived AJCC T' has 8 unique values.
Feature 'Derived AJCC N' has 8 unique values.
Feature 'Derived AJCC M' has 3 unique values.
Feature 'RX Summ--Surg Prim Site' has 19 unique values.
Feature 'RX Summ--Surg Oth Reg' has 7 unique values.
Feature 'Reason no cancer - directed surgery' has 8 unique values.
Feature 'Regional nodes examined ' has 94 unique values.
Feature 'Survival months' has 180 unique values.
Feature 'Vital status recode' has 2 unique values.
Feature 'Total number of benign tumors' has 3 unique values.
Feature 'Total number of malignant tumor' has 8 uniqu

In [4]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

# Create an instance of the LabelEncoder and OneHotEncoder
le = LabelEncoder()
ohe = OneHotEncoder()

# Apply LabelEncoder to your target variable
y_train = le.fit_transform(y_train)

# Apply OneHotEncoder to your entire dataset (both categorical and numerical)
x_train_encoded = ohe.fit_transform(x_train)
x_train_encoded = x_train_encoded.toarray()  # Convert to array if needed

# Apply StandardScaler to your data
sc = StandardScaler()
x_train_scaled = sc.fit_transform(x_train_encoded)

# Split your data into training, testing, and validation sets
x_train, x_test, y_train, y_test = train_test_split(x_train_scaled, y_train, test_size=0.30, random_state=42)
x_valid, x_test, y_valid, y_test = train_test_split(x_test, y_test, test_size=0.50, random_state=42)



<h1 id="aan-architecture" style="color:#024434; background:#f4684b; border:0.5px dotted;">
    <center>Artificial Neural Network Architecture
        <a class="anchor-link" href="#aan-architecture" target="_self">¶</a>
    </center>
</h1>

![ANN%20Arch.png](https://i.imgur.com/T3sNW2l.png)

<h1 id="aan" style="color:#024434; background:#f4684b; border:0.5px dotted;">
    <center>Artificial Neural Network Implementation
        <a class="anchor-link" href="#aan" target="_self">¶</a>
    </center>
</h1>

In [5]:
def sigmoid(x):
    return 1/(1 + np.exp(-x))

def relu(x):
    return np.maximum(0, x)

def forward(x, w, activation):
    return activation(np.matmul(x, w))

def accuracy_fn(y, y_hat):
    return (np.where(y == y_hat)[0].size / y_hat.size)

def predict(x, y_hat, weights, activation):
    predictions = np.zeros(shape=(x.shape[0]))

    for idx in range(x.shape[0]):
        r1 = x[idx, :]
        for curr_weights in weights:
            r1 = forward(r1, curr_weights, activation)
        predictions[idx] = np.where(r1 == np.max(r1))[0][0]

    accuracy = accuracy_fn(predictions, y_hat)
    return accuracy, predictions

def fitness(x, y_hat, weights, activation):
    accuracy = np.empty(shape=(weights.shape[0]))
    for idx in range(weights.shape[0]):
        accuracy[idx], _ = predict(x, y_hat, weights[idx, :], activation)
    return accuracy

### Vector-Matrix Manipulations

the provided code defines two functions, mat_to_vector and vector_to_mat, which are used to convert between matrix representations of neural network weights and vector representations. These functions are typically used in the context of evolutionary algorithms when optimizing neural network weights. These functions are useful in the context of genetic algorithms or other optimization techniques where the weights of neural networks are represented as genomes and evolved over generations. The matrix-to-vector conversion allows easier manipulation of genomes, and the vector-to-matrix conversion enables the reconstruction of neural network weight matrices for evaluation and further evolution.

In [6]:
def mat_to_vector(mat_pop_weights):
    weights_vector = []
    for idx in range(mat_pop_weights.shape[0]):
        curr_vector = []
        for layer_idx in range(mat_pop_weights.shape[1]):
            vector_weights = np.reshape(mat_pop_weights[idx, layer_idx], newshape=(mat_pop_weights[idx, layer_idx].size))
            curr_vector.extend(vector_weights)
        weights_vector.append(curr_vector)
    return np.array(weights_vector)


def vector_to_mat(vector_weights, mat_pop_weights):
    mat_weights = []
    for idx in range(mat_pop_weights.shape[0]):
        start = 0
        end = 0
        for layer_idx in range(mat_pop_weights.shape[1]):
            end = end + mat_pop_weights[idx, layer_idx].size
            curr_vector = vector_weights[idx, start:end]
            mat_layer_weights = np.reshape(curr_vector, newshape=(mat_pop_weights[idx, layer_idx].shape))
            mat_weights.append(mat_layer_weights)
            start = end
    return np.reshape(mat_weights, newshape=mat_pop_weights.shape)

<h1 id="gen-architecture" style="color:#024434; background:#f4684b; border:0.5px dotted;">
    <center>Genetic Algorithm Architecture
        <a class="anchor-link" href="#gen-architecture" target="_self">¶</a>
    </center>
</h1>

![genetic.png](https://i.imgur.com/nIPyj03.png)

<div align='center'></div>

<h1 id="gen-implementation" style="color:#024434; background:#f4684b; border:0.5px dotted;">
    <center>Genetic Algorithm Implementation
        <a class="anchor-link" href="#gen-implementation" target="_self">¶</a>
    </center>
</h1>

The provided code defines functions for genetic algorithm operations, specifically for creating a mating pool of parents, performing crossover, and introducing mutations. These operations are used in evolutionary algorithms for optimizing neural network weights. These functions are fundamental to the genetic algorithm process of evolving neural network weights over multiple generations. The mating pool selection, crossover, and mutation are essential steps for exploring and optimizing the parameter space for improved neural network performance.

In [7]:
def mating_pool(pop, fitness, num_parents):
    parents = np.empty((num_parents, pop.shape[1]))
    for parent_num in range(num_parents):
        max_fitness_idx = np.where(fitness == np.max(fitness))
        max_fitness_idx = max_fitness_idx[0][0]
        parents[parent_num, :] = pop[max_fitness_idx, :]
        fitness[max_fitness_idx] = -99
    return parents


def crossover(parents, offspring_size):
    offspring = np.empty(offspring_size)
    crossover_point = np.uint32(offspring_size[1]/2)

    for k in range(offspring_size[0]):

        parent1_idx = k%parents.shape[0]
        parent2_idx = (k+1)%parents.shape[0]

        offspring[k, 0:crossover_point] = parents[parent1_idx, 0:crossover_point]
        offspring[k, crossover_point:] = parents[parent2_idx, crossover_point:]

    return offspring


def mutation(offspring_crossover, mutation_percent):
    num_mutations = np.uint32((mutation_percent*offspring_crossover.shape[1]))
    mutation_indices = np.array(random.sample(range(0, offspring_crossover.shape[1]), num_mutations))

    for idx in range(offspring_crossover.shape[0]):
        random_value = np.random.uniform(-1.0, 1.0, 1)
        offspring_crossover[idx, mutation_indices] = offspring_crossover[idx, mutation_indices] + random_value

    return offspring_crossover

### Parameters

The provided code segment sets parameters and variables for a genetic algorithm used for optimization. It specifies the number of solutions in each population, the number of parents to be selected for mating, the total number of generations for the algorithm, and the percentage of genes to be mutated during each generation.,,,,,,,,,,,,,,,,,,,,,,,,,,, ghgdshd,

In [8]:
solution_per_population = 8
num_parents_mating = 4
num_generations = 100
mutation_percent = 0.20

In [9]:
input_shape = x_train.shape[1]
print(input_shape)
output_shape = len(np.unique(y_train))
print(output_shape)

639
2


### Initial weights

In this code segment, initial weights for a neural network are generated for a genetic algorithm optimization process. A list named initial_weights is populated with randomly initialized weight matrices for the neural network. These initial weights serve as starting points for the optimization.

In [10]:
initial_weights = []
for curr_sol in np.arange(0, solution_per_population):

    w1 = np.random.uniform(low=-0.1, high=0.1, size=(input_shape, 128))
    w2 = np.random.uniform(low=-0.1, high=0.1, size=(128, 64))
    w3 = np.random.uniform(low=-0.1, high=0.1,size=(64, output_shape))
    initial_weights.append(np.array([w1, w2, w3]))

In this code segment, the initial weights for neural networks are organized into matrix and vector representations, and arrays are created to store the best outputs and accuracies during the optimization process.

In [11]:
weights_mat = np.array(initial_weights)
weights_vector = mat_to_vector(weights_mat)

best_outputs = []
accuracies = np.empty(shape=(num_generations))


<h1 id="training" style="color:#024434; background:#f4684b; border:0.5px dotted;">
    <center>Training
        <a class="anchor-link" href="#training" target="_self">¶</a>
    </center>
</h1>

In this code segment, a genetic algorithm optimization process is executed over a specified number of generations (num_generations). During each generation, the code performs operations such as converting vectors to matrices, evaluating the fitness of the population, selecting mating parents, generating a new population through crossover and mutation, and updating the population based on the offspring created.

In [ ]:
for generation in tqdm(range(num_generations)):

    # vector to matrix
    weights_mat = vector_to_mat(weights_vector, weights_mat)

    # fitness of the population
    fit = fitness(x_train, y_train, weights_mat, activation=sigmoid)

    # assign first fitness to the array
    accuracies[generation] = fit[0]

    # selecting mating parents from pool
    parents = mating_pool(weights_vector, fit.copy(), num_parents_mating)

    # generate new population using crossover
    offspring_crossover = crossover(parents, offspring_size=(weights_vector.shape[0]-parents.shape[0], weights_vector.shape[1]))

    # adding mutation to the population
    offspring_mutation = mutation(offspring_crossover, mutation_percent=mutation_percent)

    # new population combining parents of crossver and mut
    weights_vector[0:parents.shape[0], :] = parents
    weights_vector[parents.shape[0]:, :] = offspring_mutation

  2%|▏         | 2/100 [00:06<05:08,  3.15s/it]

<h1 id="analysis" style="color:#024434; background:#f4684b; border:0.5px dotted;">
    <center>Analysis
        <a class="anchor-link" href="#analysis" target="_self">¶</a>
    </center>
</h1>

In [ ]:
weights_mat = vector_to_mat(weights_vector, weights_mat)
best_weights = weights_mat [0, :]
acc, predictions = predict(x_train, y_train, best_weights, sigmoid)
print("Accuracy of the best solution is : ", acc)

In [ ]:
plt.plot(accuracies)
plt.xlabel("Iteration", fontsize=15)
plt.ylabel("Fitness", fontsize=15)
plt.xticks(np.arange(0, num_generations+1, 100))
plt.yticks(np.arange(0, 1, 0.1))
plt.show()

### Test predictions

In [ ]:
weights_mat = vector_to_mat(weights_vector, weights_mat)
best_weights = weights_mat [0, :]
acc, predictions = predict(x_test, y_test, best_weights, sigmoid)
print("Accuracy of the best solution is : ", acc)

### Resources

https://www.linkedin.com/pulse/introduction-optimization-genetic-algorithm-ahmed-gad/ <br>
https://www.kdnuggets.com/2018/03/introduction-optimization-with-genetic-algorithm.html <br>
https://towardsdatascience.com/introduction-to-optimization-with-genetic-algorithm-2f5001d9964b <br>
https://www.springer.com/us/book/9781484241660